In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [2]:
training_logs_dir = 'evaluation_logs'
watch_dates = ['20200106', '20200123', '20200124', '20200125', '20200129', '20200214']#]#['20191118', '20191218']
contents = [] 
for watch_date in watch_dates:
    contents.append(pd.read_csv('{}.txt'.format(os.path.join(training_logs_dir, watch_date)), sep='\t'))
logs_df = pd.concat(contents, axis=0).reset_index()
logs_df.head()

,index,epoch,seed,train,strategy,accuracy,loss
0,0,1,10,True,freeze,46.922,0.022765
1,1,1,10,False,freeze,59.350,0.018236
2,2,2,10,True,freeze,68.006,0.014055
3,3,2,10,False,freeze,71.570,0.012942
4,4,3,10,True,freeze,77.188,0.010228


In [4]:
style_args = {
    'linestyle' : {
        'train': '-',
        'test': '--'
    },
    'colors': {
        'freeze': '#B61616',
        'shuffle': '#5AB616',
        'homogeneous': '#26A4C1',
        'heterogeneous': '#3626C1',
        'max_k_loss': '#ECAA1D',
        'min_k_loss': '#EC2CDB',
        'heterogeneous_max_k_loss': '#EC8E1D',
        'heterogeneous_min_k_loss': '#6BEC1D',
        'no_update_max_k_loss': '#1D49EC',
        'no_update_min_k_loss': '#A71DEC',
        'no_update_heterogeneous_max_k_loss': '#EC1D49',
        'no_update_heterogeneous_min_k_loss': '#1DECEC',
        'testupdate_max_k_loss': '#222AAA',
        'weighted_random_sampling': '#111EEE'
    }
}

---

In [5]:
def summarize(df, is_train, strategy, get_acc=True):
    '''
    @df = dataframe with the training/testing logs (epoch, seed, train, shuffle, accuracy, loss)
    @is_train = boolean whether to look at training logs
    @is_shuffle = boolean whether the data is shuffled
    @get_acc = boolean whether to get the accuracy or the loss
    '''
    subset_df = df[['epoch', 'accuracy', 'loss']].loc[(logs_df['train'] == is_train) & (logs_df['strategy'] == strategy)]
    grouped_df = subset_df.groupby('epoch')
    if get_acc:
        return {'mean': grouped_df.mean().values.T[0], 'max': grouped_df.max().values.T[0], 'min': grouped_df.min().values.T[0]}
    else:
        return {'mean': grouped_df.mean().values.T[1], 'max': grouped_df.max().values.T[1], 'min': grouped_df.min().values.T[1]}

In [6]:
def metric_over_epochs(strategies, idx, get_acc=True, data_dir='figures'):
    best_vals = {}
    for is_train in [True, False]:
        plt.close()
        plt.figure(figsize=(12,12))
        for strategy in strategies:
            # data
            metric_dict = summarize(logs_df, is_train, strategy, get_acc=get_acc)
            
            means = metric_dict['mean']
            upper_shadow = metric_dict['max']
            lower_shadow = metric_dict['min']
            idxs = range(len(means))
            
            # graph settings
            train_mode = 'train' if is_train else 'test'
            name = '{}/{}'.format(train_mode, strategy)
            best_vals[name] = max(means)
            color = style_args['colors'][strategy]
            linestyle = style_args['linestyle'][train_mode]

            # plotting
            plt.plot(idxs, means, label=name, color=color, linestyle=linestyle)
            plt.fill_between(idxs, means, upper_shadow, color=color, linestyle=linestyle, alpha=0.5)
            plt.fill_between(idxs, lower_shadow, means, color=color, linestyle=linestyle, alpha=0.5)

        plt.title('Change of {} over iterations'.format('accuracy' if get_acc else 'loss'))
        plt.xlabel('Iteration')
        if get_acc:
            plt.ylim(0, 100)
        plt.ylabel('{}'.format('Accuracy' if get_acc else 'Loss'))
        plt.legend(loc='lower right');
        plt.savefig('{}/{}_{}_{}_{}.jpg'.format(data_dir, today, idx,
                                             'accuracy' if get_acc else 'loss', 
                                             'train' if is_train else 'test'), format='jpg')
        plt.close()
        
    print("Best values:")
    print(pd.Series(best_vals))

In [7]:
strategy = 'max_k_loss'
metric_over_epochs(['shuffle', strategy], strategy, get_acc=True)
metric_over_epochs(['shuffle', strategy], strategy, get_acc=False)

Best values:
train/shuffle       99.470000
train/max_k_loss    99.106667
test/shuffle        84.550000
test/max_k_loss     78.563333
dtype: float64
Best values:
train/shuffle       0.022841
train/max_k_loss    0.023573
test/shuffle        0.019198
test/max_k_loss     0.023783
dtype: float64


In [8]:
strategy = 'no_update_max_k_loss'
metric_over_epochs(['shuffle', strategy], strategy, get_acc=True)
metric_over_epochs(['shuffle', strategy], strategy, get_acc=False)

Best values:
train/shuffle                 99.470000
train/no_update_max_k_loss    97.600000
test/shuffle                  84.550000
test/no_update_max_k_loss     78.386667
dtype: float64
Best values:
train/shuffle                 0.022841
train/no_update_max_k_loss    0.033080
test/shuffle                  0.019198
test/no_update_max_k_loss     0.034562
dtype: float64


In [9]:
strategy = 'testupdate_max_k_loss'
metric_over_epochs(['shuffle', strategy], strategy, get_acc=True)
metric_over_epochs(['shuffle', strategy], strategy, get_acc=False)

Best values:
train/shuffle                  99.470000
train/testupdate_max_k_loss    97.600000
test/shuffle                   84.550000
test/testupdate_max_k_loss     78.386667
dtype: float64
Best values:
train/shuffle                  0.022841
train/testupdate_max_k_loss    0.033080
test/shuffle                   0.019198
test/testupdate_max_k_loss     0.034562
dtype: float64


In [10]:
strategy = 'weighted_random_sampling'
metric_over_epochs(['shuffle', strategy], strategy, get_acc=True)
metric_over_epochs(['shuffle', strategy], strategy, get_acc=False)

Best values:
train/shuffle                     99.470000
train/weighted_random_sampling    76.030000
test/shuffle                      84.550000
test/weighted_random_sampling     65.263333
dtype: float64
Best values:
train/shuffle                     0.022841
train/weighted_random_sampling    0.031875
test/shuffle                      0.019198
test/weighted_random_sampling     0.033065
dtype: float64
